In [2]:
import os
import pathlib
from os import listdir

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [89]:
small_genres_df = pd.read_pickle(
    "/home/kiri/Kiri_coding/MLDS/Year project/Prediction-models-for-songs-2023/SoundOfDownloader/tracks_df.pickle"
)

In [88]:
# трек удалён из спотифая
# small_genres_df[small_genres_df['track_id'] != '1KbAfcJS3Cbrd3htDBtdcG'].to_pickle("/home/kiri/Kiri_coding/MLDS/Year project/Prediction-models-for-songs-2023/SoundOfDownloader/tracks_df.pickle")
# small_genres_df = pd.read_pickle("/home/kiri/Kiri_coding/MLDS/Year project/Prediction-models-for-songs-2023/SoundOfDownloader/tracks_df.pickle")

### Проверка наличия всех треков

In [56]:
all_tracks_ids = set(small_genres_df["track_id"])
tracks_found = []
dir_parts = pathlib.Path(
    "/home/kiri/Kiri_coding/SpotyParser/notebooks/data/small_genres_tracks_analysis"
)

for part_dir in listdir(dir_parts):
    for track in listdir(dir_parts / part_dir):
        path_to_track = dir_parts / part_dir / track
        track_id = path_to_track.stem
        tracks_found.append(track_id)

tracks_found = set(tracks_found)
tracks_left = all_tracks_ids.difference(tracks_found)

#### Удаление треков, скачанных с ошибкой

In [68]:
dir_parts = pathlib.Path(
    "/home/kiri/Kiri_coding/SpotyParser/notebooks/data/small_genres_tracks_analysis"
)

tracks_corrupted = []
for part_dir in tqdm(listdir(dir_parts)):
    for track in listdir(dir_parts / part_dir):
        path_to_track = dir_parts / part_dir / track
        track_id = path_to_track.stem
        track_pickle = pd.read_pickle(path_to_track)
        if "error" in track_pickle:
            tracks_corrupted.append([path_to_track, track_id])

  0%|          | 0/139 [00:00<?, ?it/s]

In [75]:
if tracks_corrupted:
    for path_to_track, track_id in tracks_corrupted:
        os.system(f"rm {path_to_track}")

In [91]:
all_tracks_ids = set(small_genres_df["track_id"])
tracks_found = []
dir_parts = pathlib.Path(
    "/home/kiri/Kiri_coding/SpotyParser/notebooks/data/small_genres_tracks_analysis"
)

for part_dir in listdir(dir_parts):
    for track in listdir(dir_parts / part_dir):
        path_to_track = dir_parts / part_dir / track
        track_id = path_to_track.stem
        tracks_found.append(track_id)

tracks_found = set(tracks_found)
tracks_left = all_tracks_ids.difference(tracks_found)

#### Доброска не найденных треков (дальнейшная докачка через скрипты)

In [93]:
if tracks_left:
    df_left = small_genres_df[small_genres_df["track_id"].isin(tracks_left)]
    df_left = pd.DataFrame(df_left["track_id"])
    splitted = np.array_split(df_left, 8)
    for i, df_split in enumerate(splitted):
        df_split.to_csv(f"../scripts/data/small_genres_chunks/chunk_{i+1}.csv")
else:
    print("Все треки были скачаны")

Все треки были скачаны


### Объединение треков в датасет

In [94]:
dir_parts = pathlib.Path(
    "/home/kiri/Kiri_coding/SpotyParser/notebooks/data/small_genres_tracks_analysis"
)

rows = []
for part_dir in tqdm(listdir(dir_parts)):
    for track in listdir(dir_parts / part_dir):
        to_row = []
        path_to_track = dir_parts / part_dir / track
        track_id = path_to_track.stem
        to_row.append(track_id)

        a_analysis = pd.read_pickle(path_to_track)
        trackdata = a_analysis["track"]
        keys_to_append = [
            "duration",  # in seconds
            "end_of_fade_in",
            "start_of_fade_out",
            "tempo_confidence",
            "time_signature",
            "time_signature_confidence",
            "key_confidence",
            "mode_confidence",
        ]
        for key in keys_to_append:
            to_row.append(trackdata[key])
        rows.append(to_row)

  0%|          | 0/139 [00:00<?, ?it/s]

In [102]:
analysis_df = pd.DataFrame(
    data=rows,
    columns=[
        "track_id",
        "duration_seconds",  # in seconds
        "end_of_fade_in",
        "start_of_fade_out",
        "tempo_confidence",
        "time_signature",
        "time_signature_confidence",
        "key_confidence",
        "mode_confidence",
    ],
)
analysis_df = analysis_df.drop_duplicates()

In [34]:
analysis_df.to_pickle("../../data/small_genres_audio_analysis_df.pickle")

### Объединение датасета с аудиоанализом с датасетом с аудиофичами

In [5]:
analysis_df = pd.read_pickle("../../data/small_genres_audio_analysis_df.pickle")
features_df = pd.read_pickle("../../data/small_genres_tracks_df.pickle")

In [36]:
merged_df = pd.merge(features_df, analysis_df, on="track_id")
merged_df = merged_df.drop(["time_signature_y"], axis=1).rename(
    {"time_signature_x": "time_signature"}, axis=1
)
merged_df.to_pickle("../../data/small_genres_merged_df.pickle")